In [1]:
%config Completer.use_jedi = False

In [2]:
import streamlit as st
import functools
import os, sys
import pickle
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import stats
sys.path.append("../analysis")
from pdb_util import AACODES_DICT, AACODES_DICT_upper
from collections import defaultdict
from collections import defaultdict
from tqdm import tqdm

sys.path.append("../streamlit/")
sys.path.append("./")

from streamlit_utils import DICT_PDBID_2_Atten12, DICT_PDBID_2_MELTDIST, STD_THH, \
    DICT_PDBID_2_model_out, DICT_PDBID_2_CHAINNAMES, DICT_PDBID_2_CDRS, DICT_PDBID_2_RESIDUES, \
    POSITIVE_PRED_IDS, HEAD_COUNT, get_mapping_2_4digits, read_hhb_file

from streamlit_utils import strong_givenPEP_distributedTCR_by_head, \
    strong_givenTCR_distributedPEP_by_head, general_givenPEP_distributedTCR_by_head, general_givenTCR_distributedPEP_by_head


import matplotlib.pyplot as plt

NUM2TCRORPEP = {0:'alpha', 1:'beta', 2:'peptide'}


def read_hhb_text_and_find_chain(text, chainsletters):
    ress_atoms = text[:45].replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')
    donor_res = ress_atoms[0]
    acceptor_res = ress_atoms[2]
    if donor_res.split('/')[1][:1] in chainsletters or \
            acceptor_res.split('/')[1][:1] in chainsletters:
        return text


# def hhbond_analysis_single2():
def add_resname(r):
    return AACODES_DICT_upper.get(r.get_resname(), 'X')

def add_resseq(r):
    return r.get_full_id()[3][1]

def add_4digits(r):
    res = r.get_full_id()[2]
    digits = r.get_full_id()[3][1]
    return f'{res}{digits:004d}'



def add_restype_cdr(ser):
    if ser['CDR']=='Not_CDR':
        return 3
    else:
        if ser['LargeAtten_CDR']:
            return 1
        else:
            return 2

def add_restype_pep(ser):
    if ser['Peptide']=='Not_Peptide':
        return None
    else:
        if ser['LargeAtten_Peptide']:
            return 4
        else:
            return 5

# def hhbond_stats_10patterns():

loading....


  0%|                                                                                           | 0/4 [00:00<?, ?it/s]

ending... loading....


100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:02<00:00,  1.74it/s]
25it [00:00, 55.41it/s]

2OL3 has only delta-chain and beta-chain


67it [00:01, 52.27it/s]

4QRR has only delta-chain and beta-chain
5JHD has only delta-chain and beta-chain
5JZI has only delta-chain and beta-chain


79it [00:01, 50.23it/s]
/Users/kyoheikoyama/.pyenv/versions/miniconda3-latest/envs/tcrpred/lib/python3.8/site-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [66]:

print("# Show No Hbond but Large Attention")

#########################################################################
# print("# Attention file pre-processing: ")
df_strong_attention = pd.concat(
    [pd.concat(strong_givenPEP_distributedTCR_by_head[i]) for i in range(4)])
df_strong_attention = df_strong_attention.drop_duplicates(subset=['tcr','pdbid'])
df_strong_attention_pep = pd.concat(
    [pd.concat(strong_givenTCR_distributedPEP_by_head[i]) for i in range(4)])
df_strong_attention_pep = df_strong_attention_pep.drop_duplicates(subset=['peptide','pdbid'])
df_normal_attention = pd.concat(
    [pd.concat(general_givenPEP_distributedTCR_by_head[i]) for i in range(4)])
df_normal_attention = df_normal_attention.drop_duplicates(subset=['tcr','pdbid'])

isin_strong = df_normal_attention[['tcr','pdbid']].apply(tuple, axis=1).isin(df_strong_attention[['tcr','pdbid']].apply(tuple, axis=1))
df_normal_attention = df_normal_attention[~isin_strong]

large_atten = df_strong_attention['tcr'].apply(lambda x: x[0]).value_counts()
if 'H' not in large_atten.index:
    large_atten = large_atten.append(pd.Series({'H':0}))
aa_residues = df_normal_attention['tcr'].apply(lambda x: x[0]).unique()
aa_residues_selected = [aa for aa in aa_residues if aa in large_atten.index]
small_atten = df_normal_attention['tcr'].apply(lambda x: x[0]).value_counts()


## Dedup pdbid_list
unique_pdbid_list = df_strong_attention['pdbid'].unique()

df_cdrs_and_peps = pd.DataFrame([tuple([DICT_PDBID_2_Atten12[p][0][0].index.tolist() + DICT_PDBID_2_Atten12[p][0][0].columns.tolist()])
                                 for p in unique_pdbid_list],
                                columns=['pairs'], index=unique_pdbid_list)['pairs'].apply(tuple)
unique_pdbid_list = df_cdrs_and_peps.drop_duplicates().index

######################################################################
### PDBID FOR LOOP ####################################################


vals_dict = defaultdict(list)


# csv_list_nohbond = list()
csv_dict_nohbond = dict()
DICT_AAtype_count = dict()

for pdbid in reversed(unique_pdbid_list):
    residues = DICT_PDBID_2_RESIDUES[pdbid]
    chains = DICT_PDBID_2_CHAINNAMES[pdbid.upper()]
    chainsletters = ''.join(chains)
    
    dfhhb = read_hhb_file(pdbid)
    donors_n_acceptors = [read_hhb_text_and_find_chain(row.item(), chainsletters) for i, row in dfhhb.iterrows()]
    donors_n_acceptors = [a for a in donors_n_acceptors if a is not None]
    
    bond_tuple_fromto = []
    for da in donors_n_acceptors:
        a = da.replace('  ',' ').replace('  ',' ').replace('  ',' ').replace('  ',' ').split(' ')
        from_d = a[0].split('/')[1].split('-')[0][:5]
        to_d = a[2].split('/')[1].split('-')[0][:5]
        bond_tuple_fromto.append((from_d, to_d))
    anyhbond_related_4digits = list(set([tup[0] for tup in bond_tuple_fromto] + [tup[1] for tup in bond_tuple_fromto]))

    CDRname_2_4digits, Peptide_2_4digits, bond_list = get_mapping_2_4digits(donors_n_acceptors, pdbid)

    D4digits_2_CDR = {v:k for k,v in CDRname_2_4digits.items()}
    D4digits_2_Peptide = {v:k for k,v in Peptide_2_4digits.items()}

    df_strong_temp = df_strong_attention[df_strong_attention.pdbid == pdbid].copy()
    df_strong_temp['tcr_residue'] = df_strong_temp['tcr'].map(CDRname_2_4digits)
    df_strong_temp['pep_residue'] = df_strong_temp['peptide'].map(Peptide_2_4digits)

    df_strong_temp_peptide = df_strong_attention_pep[df_strong_attention_pep.pdbid == pdbid].copy()

    def add_info(df):
        df['AA resname'] = df[0].apply(add_resname)
        df['Resseq'] = df[0].apply(add_resseq)
        df['4digits'] = df[0].apply(add_4digits)
        df['CDR'] = df['4digits'].map(D4digits_2_CDR)
        df['CDR'] = df['CDR'].fillna('Not_CDR')
        df['LargeAtten_CDR'] = df['CDR'].isin(df_strong_temp['tcr'].values)
        df['Peptide'] = df['4digits'].map(D4digits_2_Peptide)
        df['Peptide'] = df['Peptide'].fillna('Not_Peptide')
        df['LargeAtten_Peptide'] = df['Peptide'].isin(df_strong_temp_peptide['peptide'].values)
        df['has_bond'] = df['4digits'].isin(anyhbond_related_4digits)
        return df

    df_alpha = pd.DataFrame([residues[0]]).T
    df_beta = pd.DataFrame([residues[1]]).T
    df_pep = pd.DataFrame([residues[2]]).T
    df_alpha = add_info(df_alpha)
    df_alpha['chain_name'] = chains[0]
    df_beta = add_info(df_beta)
    df_beta['chain_name'] = chains[1]
    df_pep = add_info(df_pep)
    df_pep['chain_name'] = chains[2]

    df_alpha['restype'] = df_alpha.apply(add_restype_cdr, axis=1)
    df_beta['restype'] = df_beta.apply(add_restype_cdr, axis=1)
    df_pep['restype'] = df_pep.apply(add_restype_pep, axis=1)

    df_very_all = pd.concat([df_pep, df_alpha, df_beta])
    
    
    df_bond_all_4digits = pd.DataFrame(anyhbond_related_4digits, columns=['4digits'])
    df_bond_all_4digits = pd.merge(df_bond_all_4digits, df_very_all, on=['4digits'], how='inner')

    def get_acceptor(donor):
        return [a for (d,a) in bond_tuple_fromto if d==donor]

    def get_donor(aceptor):
        return [d for (d,a) in bond_tuple_fromto if a==aceptor]

    df_bond_all_4digits['bonding_to'] = df_bond_all_4digits['4digits'].apply(get_acceptor)
    df_bond_all_4digits['bonding_from'] = df_bond_all_4digits['4digits'].apply(get_donor)
    df_bond_all_4digits['bonding_tofrom'] = df_bond_all_4digits['bonding_to'] + df_bond_all_4digits['bonding_from']
    BOND_MAPPING = {k:v for k,v in zip(df_bond_all_4digits['4digits'], df_bond_all_4digits['bonding_tofrom'])}
    df_very_all['bonding_tofrom'] = [np.array([]) for _ in range(len(df_very_all))]
    df_very_all['bonding_tofrom'] = df_very_all['4digits'].map(BOND_MAPPING)
    df_very_all.loc[df_very_all['bonding_tofrom'].isna(), 'bonding_tofrom'] = [[None] for _ in range(df_very_all['bonding_tofrom'].isna().sum())]
    
    df_strong_temp['tcr_residue'].isin(df_bond_all_4digits['4digits'])
#     print(df_strong_temp.loc[~df_strong_temp['tcr_residue'].isin(df_bond_all_4digits['4digits'])].tcr.tolist()) 
    
    print(pdbid,":", 
          df_strong_temp.loc[
              ~df_strong_temp['tcr_residue'].isin(df_bond_all_4digits['4digits'])].tcr_residue.tolist())
    
    csv_dict_nohbond.update({pdbid: 
                             list(df_strong_temp.loc[~df_strong_temp['tcr_residue'].isin(df_bond_all_4digits['4digits'])].tcr_residue.tolist())})
    
    if len(csv_dict_nohbond[pdbid])==0:
        continue
    else:
        nobond_AAs = csv_dict_nohbond[pdbid]
        df_strong_but_no_bond = df_strong_temp[df_strong_temp.tcr_residue.isin(nobond_AAs)]
        display(df_strong_but_no_bond)
        
        DICT_AAtype_count[pdbid] = df_strong_but_no_bond.tcr.tolist()
        pass
    
    if pdbid=='5WKF':
        break
    
    
    

# Show No Hbond but Large Attention
6EQA : ['E0099', 'E0101']


,peptide,tcr,value_x,value_y,pdbid,head,tcr_residue,pep_residue
167,L_6,G_20,5.733288,0.401790,6EQA,3,E0099,C0008
220,V_8,G_22,17.537863,0.429731,6EQA,3,E0101,C0010


3VXU : []
4OZG : ['E0116']


,peptide,tcr,value_x,value_y,pdbid,head,tcr_residue,pep_residue
40,E_4,L_9,19.860783,0.806244,4OZG,1,E0116,I0006


4OZF : ['G0116']


...

,peptide,tcr,value_x,value_y,pdbid,head,tcr_residue,pep_residue
144,S_6,G_22,12.234696,0.603522,4MJI,2,E0103,C0007


3PQY : ['D0110', 'D0115']


,peptide,tcr,value_x,value_y,pdbid,head,tcr_residue,pep_residue
116,N_4,S_6,6.123177,0.620989,3PQY,1,D0110,C0005
21,A_7,T_11,18.662855,0.428112,3PQY,3,D0115,C0008


5WKF : ['D0097']


,peptide,tcr,value_x,value_y,pdbid,head,tcr_residue,pep_residue
72,I_6,M_8,12.273016,0.40412,5WKF,0,D0097,C0007


In [76]:
DICT_AAtype_count

{'6EQA': ['G_20', 'G_22'],
 '4OZG': ['L_9'],
 '4OZF': ['L_9'],
 '5WKH': ['M_8', 'G_24'],
 '1D9K': ['F_11', 'F_25', 'Q_18'],
 '5D2L': ['Q_7', 'F_10'],
 '4QOK': ['E_23', 'G_20'],
 '5NHT': ['D_7'],
 '2NX5': ['G_6', 'F_28'],
 '4OZH': ['V_2', 'K_9'],
 '3MBE': ['L_11'],
 '2Z31': ['K_10'],
 '3QIU': ['K_9'],
 '1U3H': ['F_27'],
 '4JRX': ['K_10'],
 '2J8U': ['F_9', 'V_22'],
 '3VXR': ['T_21'],
 '4P2R': ['D_24'],
 '4P2Q': ['D_24'],
 '4MJI': ['G_22'],
 '3PQY': ['S_6', 'T_11'],
 '5WKF': ['M_8']}

In [77]:
a = pd.value_counts([v.split('_')[0] for vlist in  DICT_AAtype_count.values() for v in vlist]).to_dict()
    

In [89]:
for k,v in zip(a.keys(), np.array(list(a.values())) / sum(a.values())):
    print(k,':',v)

G : 0.1875
F : 0.1875
K : 0.125
L : 0.09375
D : 0.09375
M : 0.0625
Q : 0.0625
V : 0.0625
T : 0.0625
E : 0.03125
S : 0.03125


In [95]:
dfa = pd.DataFrame(zip(a.keys(), np.array(list(a.values())) / sum(a.values()))).rename(
    columns={0:'AAType', 1:'Ratio_noHbond_largeAtten'})

In [100]:
dfb = pd.read_csv('/Users/kyoheikoyama/Downloads/small_atten.csv').rename(
    columns={'Unnamed: 0':'AAType', 'tcr':'Ratio_in_alldata'})

In [102]:
dfratio = pd.merge(dfa, dfb, on='AAType', how='left')

In [106]:
dfratio['Ratio_noHbond_largeAtten/Ratio_in_alldata'] =  dfratio.Ratio_noHbond_largeAtten/dfratio.Ratio_in_alldata


In [108]:
dfratio

,AAType,Ratio_noHbond_largeAtten,Ratio_in_alldata,Ratio_noHbond_largeAtten/Ratio_in_alldata
0,G,0.18750,0.119025,1.575295
1,F,0.18750,0.105904,1.770465
2,K,0.12500,0.012184,10.259615
3,L,0.09375,0.073102,1.282452
4,D,0.09375,0.018744,5.001563
5,M,0.06250,0.004686,13.337500
6,Q,0.06250,0.033739,1.852431
7,V,0.06250,0.022493,2.778646
8,T,0.06250,0.064667,0.966486
9,E,0.03125,0.035614,0.877467
